## Genre Classification Model
 

**Objective.** Construct a classification model that can accurately predict the genre of a song using a dataset containing Spotify track information, including artist details and genres. The model will utilize the audio features and potentially the lyrics of each track to make its predictions. Additionally, it will be trained using the artist genre list as well as a separate dataset of tracks that have been labeled with their respective genres.


Constructing a classification model to predict the genre of a song involves several steps, including data preprocessing, feature engineering, model selection, training, and evaluation. Here's a step-by-step guide to achieve this:




In [1]:
import pandas as pd
import warnings 
warnings.simplefilter("ignore")
#from genre_seed import track_features

genres_df = pd.read_csv('../assets/data/genres_v2.csv')
genres_v2 = pd.read_csv('../assets/data/genre_seeds.csv')
#genres_v2=genres_v2.drop_duplicates(subset=['track_id']).reset_index()

In [2]:
genres_v2['genre'].value_counts()

acoustic    110
pop         109
emo         108
happy       108
sad         107
punk        107
romance     106
grunge      106
hip-hop     105
chill       105
dance       103
piano       103
indie       102
rock        101
edm          77
techno       59
r-n-b        58
Name: genre, dtype: int64

### Step 1: Data Collection
Gather the necessary datasets:
- **Spotify API**: Use the Spotify API to collect track information, including audio features (e.g., danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo).
- **Artist Genre Data**: Gather genres associated with each artist from Spotify's API.
- **Lyrics Data**: Includes the lyrics of the tracks (optional but can enhance the model's performance).
- **Genre Labels**: Obtain a labeled dataset of tracks with their respective genres for training.


We want to try and predict a song's genre based off of these audio features. Spotify provides a "genre seed:" an array of genres associated with the song used for the recommendation function of the API. We use the API to search for the top 1000 songs in a given genre, pull the audio features for each song, and add on the genre label.


In [3]:
df = genres_v2[['name', 'track_id', 'artist', 'popularity', 'artist_genres', 
                'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
                'loudness', 'speechiness', 'tempo', 'valence', 'key', 'mode','time_signature', 'genre']]

# Group by 'Category' and describe
grouped_description = df.groupby('genre').describe()
df.describe()

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
count,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000,1674.000000
mean,27.680406,0.228153,0.557579,0.654587,0.113044,0.188549,-7.521858,0.079652,122.751480,0.441805,5.263441,0.648148,3.931302
std,27.194546,0.312398,0.167096,0.244701,0.263844,0.152054,4.460943,0.081385,28.238078,0.232465,3.607028,0.477691,0.381192
min,0.000000,0.000002,0.062100,0.001500,0.000000,0.021500,-41.446000,0.022600,42.646000,0.027500,0.000000,0.000000,1.000000
25%,0.000000,0.005390,0.444000,0.489000,0.000000,0.095725,-9.013000,0.035425,100.149750,0.256250,2.000000,0.000000,4.000000
50%,26.000000,0.056100,0.555000,0.707500,0.000046,0.126000,-6.326000,0.048200,123.467500,0.417000,6.000000,1.000000,4.000000
75%,53.000000,0.357750,0.676000,0.855000,0.012950,0.230750,-4.653750,0.084075,140.063750,0.613750,8.000000,1.000000,4.000000
max,83.000000,0.995000,0.969000,0.996000,0.978000,0.988000,-1.264000,0.578000,214.008000,0.980000,11.000000,1.000000,5.000000


---------------------------------


### Step 2: Data Preprocessing
Clean and preprocess the data to prepare it for model training.
- **Handle Missing Values**: Remove or impute missing data.
- **Normalize/Standardize Features**: Normalize or standardize the audio features to ensure they are on a similar scale.
- **Text Preprocessing for Lyrics**: Tokenize, remove stopwords, and potentially use techniques like TF-IDF or word embeddings for the lyrics.





In [4]:
# Normalize audio features
from sklearn.preprocessing import StandardScaler

# Define audio features
audio_features = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness',
                  'instrumentalness', 'liveness', 'valence', 'tempo']

# Fit and transform audio features
scaler = StandardScaler()
df[audio_features] = scaler.fit_transform(df[audio_features])

------

### Step 3. Feature Engineering
Create features from the available data.
- **Audio Features**: Use the provided audio features.
- **Artist Genre Encoding**: Encode categorical features (i.e. genres) using one-hot encoding or other suitable methods.
- **Lyrics Features**: Convert lyrics into numerical features using techniques like TF-IDF, Word2Vec, or BERT embeddings.
    - Extract additional features from lyrics (e.g., sentiment analysis, topic modeling).


#### Label Encoder

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()  # Encode the target variable
df['genre_encoded'] = le.fit_transform(df['genre'])

Combine and arrange the data to create a final dataset for training the model.


In [6]:
X = df[['danceability', 'energy', 'key', 'loudness', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
y = df[['genre_encoded']]

-----------------------

### Step 4: Model Selection and Training
Choose an appropriate classification algorithm and train the model.
- **Algorithms**: Consider using algorithms like Random Forest, Gradient Boosting, Support Vector Machine (SVM), or Neural Networks.
- **Cross-Validation**: Use cross-validation to tune hyperparameters and avoid overfitting.


#### Model Training 

The following code divides a dataset into training and testing subsets. It divides the input variables and target variables into 80% training and 20% testing groups at random. The descriptive statistics of the training data are then outputted to aid in data exploration and the identification of possible problems.

In [7]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) # test_size=0.2

#### Model Fitting

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from prettytable import PrettyTable

In [9]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, C=0.5, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_split=5, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(max_depth=7, min_samples_split=5, random_state=42),
    "Gradient Boost": GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, random_state=42),
    "XGB": XGBClassifier(n_estimators=300, random_state=42)
}

-------

### Step 6: Model Evaluation
Evaluate the model's performance using appropriate metrics.
- **Metrics**: Use accuracy, precision, recall, F1-score, and confusion matrix to assess the model.
- **Validation Set**: Use a separate validation set to test the model's generalization ability.


We can evaluate the performance of different models using multiple criteria: Accuracy, ROC-AUC, Precision, Recall, and F1-Score. Based on the provided metrics, let's analyze the performance of each model to determine which one is the best:

In [10]:
# Initialize an empty list to store the results
results = []

# Train models and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None

    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

    results.append({
        "Model": name, 
        "Accuracy": accuracy_score(y_test, y_pred), 
        "ROC-AUC": roc_auc_score(y_test, y_prob, multi_class='ovr') if y_prob is not None else None,
        "Precision": report['weighted avg']['precision'], 
        "Recall": report['weighted avg']['recall'], 
        "F1-Score": report['weighted avg']['f1-score']
    })

In [11]:
# Convert results to a DataFrame
df = pd.DataFrame(results, columns=["Model", "Accuracy", "ROC-AUC", "Precision", "Recall", "F1-Score"])

# Apply styling
props = 'font-family: "Roboto"; color: #e83e8c; font-size:13px; font-weight: 800;text-transform:uppercase;'
props2 = 'font-family: "Roboto"; color: black; font-size:12px; font-weight: 400;'

styled_df = (df.style
             .set_table_styles([{'selector': 'td.col0', 'props': props},
                                {'selector': 'td', 'props': props2}])
             .highlight_max(props='color:white !important;background-color:#d23d75;font-weight:700 !important;font-size:13px !important;',
                            axis=0, subset=['Accuracy', 'ROC-AUC', 'Precision', 'Recall', 'F1-Score']))

# Display the styled DataFrame
styled_df

,Model,Accuracy,ROC-AUC,Precision,Recall,F1-Score
0,Logistic Regression,0.291169,0.812339,0.286944,0.291169,0.272028
1,Random Forest,0.329356,0.832776,0.335009,0.329356,0.305644
2,SVM,0.298329,0.816533,0.287245,0.298329,0.278114
3,Neural Network,0.288783,0.803341,0.282435,0.288783,0.281682
4,KNN,0.233890,0.669295,0.226509,0.233890,0.222153
5,Decision Tree,0.272076,0.739925,0.271790,0.272076,0.250733
6,Gradient Boost,0.315036,0.814781,0.328956,0.315036,0.313637
7,XGB,0.286396,0.781179,0.289853,0.286396,0.285033


#### Conclusion

Considering all metrics, **Random Forest** appears to be the best model overall due to its high performance across multiple metrics (accuracy and ROC-AUC), while the **Neural Network** also performs well with the highest precision and F1-Score.

These results suggest that the Random Forest, Neural Network, and SVM models are the most effective for this specific task of predicting the genre of a song using the given dataset.


<!--
Considering these metrics, the **Gradient Boosting** model is the best performing model overall. It has the highest accuracy, precision, recall, and F1-score, making it the most balanced model in terms of performance across different evaluation metrics.-->

-------------

### Step 7: Model Deployment
Deploy the model for practical use.
- **Save the Model**: Save the trained model using libraries like joblib or pickle.
- **API Creation**: Create an API using Flask or FastAPI to make predictions on new data.


In [12]:
import joblib

# Train the Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_split=5, random_state=42)
random_forest_model.fit(X_train, y_train)

# Save scaler + model for future use
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(random_forest_model, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [13]:
# Evaluate the model
y_pred = random_forest_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.32935560859188545


#### Applying Model to New Data

In [14]:
# Load the trained models + scaler
random_forest_model = joblib.load('random_forest_model.pkl')
scaler = joblib.load('scaler.pkl')

In [15]:
# Load new data
df_new = pd.read_csv("../assets/data/all_tracks+lyrics.csv")

# Extract relevant columns
new_data = df_new[['name', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 
                   'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

# Preprocess new data
X_new_data = new_data.drop(columns=['name'])
X_new_data_scaled = scaler.transform(X_new_data)

In [16]:
# Make predictions
predictions = random_forest_model.predict(X_new_data_scaled)
probabilities = random_forest_model.predict_proba(X_new_data_scaled)

# Decode the predicted genre labels
predictions_label = le.inverse_transform(predictions)

# Display predictions
new_data['Predicted Genre'] = predictions
new_data['Predicted Genre Label'] = predictions_label
new_data['Prediction Probabilities'] = probabilities.tolist()

new_data[['name', 'Predicted Genre Label', 'Predicted Genre', 'Prediction Probabilities']]

,name,Predicted Genre Label,Predicted Genre,Prediction Probabilities
0,Espresso,pop,10,"[0.04832254153187096, 0.12619269101055017, 0.0..."
1,BIRDS OF A FEATHER,indie,8,"[0.062251132860993375, 0.1350221101568469, 0.1..."
2,I Had Some Help (Feat. Morgan Wallen),happy,6,"[0.029337791261863377, 0.07129928471933508, 0...."
3,Too Sweet,happy,6,"[0.03110784942385244, 0.07207603026340323, 0.1..."
4,MILLION DOLLAR BABY,pop,10,"[0.021188299219084113, 0.06175288340078473, 0...."
5,we can't be friends (wait for your love),pop,10,"[0.06131610996839342, 0.09285144407448995, 0.0..."
6,A Bar Song (Tipsy),pop,10,"[0.06472761794832327, 0.1272680424393216, 0.02..."
7,LUNCH,dance,2,"[0.057410671030815943, 0.07659489738600524, 0...."
8,Fortnight (feat. Post Malone),sad,15,"[0.1634002656168661, 0.13524188930269576, 0.00..."
9,Beautiful Things,romance,14,"[0.06134736364973052, 0.08543240103706232, 0.0..."


-----